# Computational Graphs &mdash; Ví dụ bằng `torch`

Nhóm 8 xin giới thiệu tới các bạn cách xây dựng đồ thị tính toán bằng PyTorch.

Dưới đây là hai cách khác nhau để xây dựng một đồ thị tính toán.

In [1]:
import torch

Bạn cần download và install `torch` để sử dụng.

Bạn cũng có thể không download thư viện mà dùng Google Colab để mở và chạy notebook này, tuy nhiên lúc này cần kết nối Internet.

Computational Graph được xây dựng và thực thi trong PyTorch như thế nào ?
- https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/
- https://pytorch.org/blog/how-computational-graphs-are-executed-in-pytorch/

In [2]:
# Data type
dtype = torch.float

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Current device:', device)
if device.type == 'cuda':
    print('Current device name:', torch.cuda.get_device_name())
    print('CUDA device count:', torch.cuda.device_count())

Current device: cpu


## Ví dụ 1 : Tự định nghĩa một hàm số

Ta định nghĩa một hàm số $f$ theo các bước sau :
- Tạo class cho $f$. Class này kế thừa `torch.autograd.Function`.
- Ghi đè hàm `forward`. Hàm này định nghĩa cách tính giá trị của nút áp dụng $f$ từ (các) đầu vào.
- Ghi đè hàm `backward`. Hàm này định nghĩa cách tính đạo hàm của nút đích cuối cùng ứng với từng đầu vào.

In [3]:
class Exp(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, x):
        result = x.exp()
        ctx.save_for_backward(result)
        return result
    
    @staticmethod
    def backward(ctx, grad_outputs):
        result, = ctx.saved_tensors
        return grad_outputs * result

Trên đây là một cách định nghĩa hàm số $f(x) = e^{x}$.

Trong hàm `forward`, ta tính giá trị của hàm ứng với $x$ (dòng `result = x.exp()`).

Trong hàm `backward`, ta tính đạo hàm của nút đích ứng với $x$. Vì

$$\frac{\partial{f(x)}}{\partial{x}} = e^{x}$$

nên ta có thể định nghĩa `backward` như trên.

In [4]:
# Let x = 0.5
x = torch.tensor(0.5, dtype=dtype, device=device, requires_grad=True)
print('x =', x)

# Compute y = f(x)
y = Exp.apply(x)
print('f(x) =', y)

# Compute dy/dx
x.grad = None
y.backward()
print('df(x)/dx =', x.grad)

x = tensor(0.5000, requires_grad=True)
f(x) = tensor(1.6487, grad_fn=<ExpBackward>)
df(x)/dx = tensor(1.6487)


Trong ví dụ trên, $x$ là một tensor mang giá trị vô hướng. 

Thông số `requires_grad=True` thông báo rằng ta cần tính đạo hàm ứng với tensor này. 

## Ví dụ 2 : Sử dụng các hàm mặc định

Xét
$$f(x, A) = x^\top A\;x$$
trong đó $x$ là vector dọc, $A$ là ma trận hai chiều. Giả sử các phép toán luôn có nghĩa.

Trong ví dụ này, ta có thể tính toán giá trị và đạo hàm theo cách đơn giản hơn nhiều.

In [5]:
# Let x = [[0],
#          [5]]
# and A = [[0  1]
#          [1  2]]

x = torch.tensor([0, 5], dtype=dtype, device=device, requires_grad=True).reshape((-1, 1))
A = torch.tensor([[0, 1], [1, 2]], dtype=dtype, device=device, requires_grad=True)
print('x')
print(x)
print('A')
print(A)

x
tensor([[0.],
        [5.]], grad_fn=<ReshapeAliasBackward0>)
A
tensor([[0., 1.],
        [1., 2.]], requires_grad=True)


Đầu tiên ta tạo các tensor cho $x$ và $A$ (chú ý `shape` của vector). Gán cho chúng một giá trị.

In [6]:
# Compute y = f(x, A)

y = x.T.mm(A).mm(x)
print('y = f(x, A) =', y.item())

y = f(x, A) = 50.0


Để tính $y = f(x, A)$, ta chỉ cần gán `y = x.T.mm(A).mm(x)`. Lúc này hàm `forward` mặc định được gọi.

Biết rằng `mm` là hàm tính phép nhân giữa hai ma trận, `x.T` là ma trận chuyển vị của `x`.

In [22]:
# Compute dy/dA, dy/dx

for var in (x, A):
    var.retain_grad()
    var.grad = None
y.backward(retain_graph=True)
print('dy/dx')
print(x.grad)
print('dy/dA')
print(A.grad)

dy/dx
tensor([[10.],
        [20.]])
dy/dA
tensor([[ 0.,  0.],
        [ 0., 25.]])


Để tính đạo hàm của `y` ứng với `x` và `A`, ta gán thông số `grad` của mỗi đầu vào bằng `None` và gọi `y.backward()` để tính đạo hàm. Đạo hàm khi này được lưu thẳng vào thông số `grad` của mỗi đầu vào.

Hàm `retain_grad()` của tensor thông báo rằng tensor đó cần được tính đạo hàm, dù có thể không phải nút lá.

Các định nghĩa về đạo hàm của các phép toán được áp dụng (nhân và chuyển vị ma trận) đã được định nghĩa sẵn bởi PyTorch, nên khác với ví dụ 1, ta không cần định nghĩa lại nữa.